In [2]:
pip install boto3

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 131 kB 29.2 MB/s eta 0:00:01
     |████████████████████████████████| 79 kB 18.0 MB/s eta 0:00:01
     |████████████████████████████████| 8.4 MB 96.4 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [1]:
import requests
import re
import time
import json
import sys
import boto3

In [2]:
s3 = boto3.client('s3')
s3_resource = boto3.resource('s3')
bucket_resource = s3_resource.Bucket('macs30123-bills')

In [3]:
API_KEY = 'nt5nhSpwSqMbrGJ7hcsBkXFI1mfk80X0fexbnt45'

In [69]:
get_offset('114')

0.4716981132075472% done
1.140435536702474e-06 minutes elapsed
0.9433962264150944% done
0.15585529406865437 minutes elapsed
1.4150943396226416% done
0.3080597877502441 minutes elapsed
1.8867924528301887% done
0.4711043079694112 minutes elapsed
2.358490566037736% done
0.6487506469090779 minutes elapsed
2.8301886792452833% done
0.7980081558227539 minutes elapsed
3.30188679245283% done
0.9551332751909892 minutes elapsed
Reached limit at offset 1000
Reached limit at offset 1100
Reached limit at offset 1200
Reached limit at offset 1300
Reached limit at offset 1400
Reached limit at offset 1500
Reached limit at offset 1600
Reached limit at offset 1700
Reached limit at offset 1800
Reached limit at offset 1900
Reached limit at offset 2000
Reached limit at offset 2100
Reached limit at offset 2200
Reached limit at offset 2300
Reached limit at offset 2400
Reached limit at offset 2500
Reached limit at offset 2600
Reached limit at offset 2700
Reached limit at offset 2800
Reached limit at offset 2900

KeyboardInterrupt: 

In [67]:
def get_offset(congress="116"):
    '''
    The govinfo API allows us to query 100 'collections' at a time. For a given
    congressional year, we need to call get_bill_ids() x times, depending on the
    number of total bills from that year. For example, Congress 116 had 20450,
    so we need to call 20450/100 = 205 times, changing the offset flag each time.

    This function iterates through docClass, billVersion, and offset values and
    calls get_bill_ids() for each combination.

    Inputs:
        congress (str) - the congress number we want to query (default is 116)
    Returns:
        get_package(all_bill_ids, congress) - This function calls the
            get_package() function which will get all bill texts for the list
            of all_bill_ids

    This function also writes all all_bill_ids to json file.
    '''

    all_bill_ids = {congress: []}

    docClass = ['s', 'hr', 'hres', 'sconres']
    billVersion = ['as', 'ash', 'ath', 'ats', 'cdh', 'cds', 'cph', 'cps', 'eah',
                   'eas', 'eh', 'eph', 'phs', 'enr', 'es', 'fah', 'fph', 'fps',
                   'hdh', 'hds', 'ih', 'iph', 'ips', 'is', 'lth', 'lts', 'oph',
                   'ops', 'pav', 'pch', 'pcs', 'pp', 'pap', 'pwah', 'rah',
                   'ras', 'rch', 'rcs', 'rdh', 'rds', 'reah', 'res', 'renr',
                   'rfh', 'rfs', 'rh', 'rih', 'ris', 'rs', 'rth', 'rts', 'sas',
                   'sc']

    start_time = time.time()
    for num2, dclass in enumerate(docClass): # 4 loops
        for num3, version in enumerate(billVersion): # 53 loops
            # dclass = "hr"
            # version = "ih"
            print(f"{((len(billVersion)*num2)+(num3 + 1))/(len(docClass)*len(billVersion))*100}% done")
            now = time.time()
            # print(f"{((num2 * num3) / 212)*100}% done")
            print(f"{(now - start_time)/60} minutes elapsed")
            # being limited at 10000 for all years
            # https://github.com/usgpo/api/issues/19#issuecomment-428292313
            for x in range(0, 10000, 100): # 100 loops
                # print(x)
                subset_bill_ids = get_bill_ids(offset=x,
                                                        congress=congress,
                                                        docClass=dclass,
                                                        billVersion=version)
                if subset_bill_ids:
                    all_bill_ids[congress].extend(subset_bill_ids)
    # print(all_bill_ids)
    # NEEDS TO GO IN S3
    # bucket_resource.put_object(all_bill_ids)
    print("put in bucket")
    bucket_resource.put_object(Key=f'{congress}_ids', Body=json.dumps(all_bill_ids))
    print(len(all_bill_ids[congress]), "gathered")
    print("check bucket")
    # with open(f'bill_ids/{congress}_ids.json', 'w') as outfile:
        # json.dump(all_bill_ids, outfile)
    # return get_package(all_bill_ids, congress)

In [68]:
def get_bill_ids(lastModifiedStartDate='1990-05-13T02:22:08Z',
                 offset=0,
                 pageSize=100,
                 congress='116',
                 docClass='s',
                 billVersion='is'):
    '''
    Given an offset value and congress number, this function makes a govinfo
    API call to request a list of 'collections' (bill ids). We will filter for
    bill ids that contain the word 'climate' or 'Climate' in the title.

    Inputs:
        lastModifiedStartDate (str) - This is the start date and time in
            ISO8601 format (yyyy-MM-dd'T'HH:mm:ss'Z')
        offset (int) - This is the starting record you wish to retrieve
        pageSize (int) - The number of records you would like to return within
            a given request
        congress (str) - congress number (116 default)
        docClass (str) - bill/collection categories (s, hr, hres, sconres)
        billVersion (str) - the bill version (there are 53 possible types)
    Returns:
        climate_bill_ids (list) - list of bill_ids that contain 'climate' or
            'Climate' in the title
        returns None if the API call results in bad status code
    '''

    url = f'https://api.govinfo.gov/collections/BILLS/{lastModifiedStartDate}?offset={offset}&pageSize={pageSize}&congress={congress}&docClass={docClass}&billVersion={billVersion}&api_key={API_KEY}'
    PARAMS = {'headers': 'accept: application/json'}

    r = requests.get(url=url, params=PARAMS)
    if re.search(r"OVER_RATE_LIMIT", r.content.decode('utf8')):
        print("Reached limit at offset", offset)
        return None
    elif r.status_code == 200:
        data = r.json()
        packages = data['packages']
        
        # climate_bill_ids = []
        # for package in packages:
            # if re.search(r"(C|c)limate", package['title']):
                # climate_bill_ids.append(package['packageId'])
        # climate_bill_ids_dict = {congress: climate_bill_ids}
        
        # print(climate_bill_ids)
        return packages
    return None

In [41]:
def get_package(all_bill_ids, congress, version):
    '''
    Given a list of bill ids (pre filtered for those that contain 'climate' in
    title), this function makes a govinfo API call to request the bill text.

    Inputs:
        all_bill_ids (dict) - A dictionary with the congress number as
            the key, and a list of climate_bill_ids (list of bill ids) as the
            value
        congress (str) - the congress number
    Returns:
        This function dumps a dictionary of dictionaries to
        climate_bills/{congress}_bills.json. The dictionary key is the congress
        number. The dictionary value is another nested dictionary. The nested
        dictionary keys are the bill ids. The nested dictionary values are the
        bill text.
    '''
    all_bills = {congress: {}}
    # all_bills = {'116': {}, '115': {}, '114': {}, '113': {}, '112': {},
    #             '111': {}, '110': {}, '109': {}, '108': {}, '107': {},
    #             '106': {}, '105': {}, '104': {}, '103': {}}

    # for congress in all_bill_ids.keys():
    # for num, package in enumerate(all_bill_ids[congress]):
    for num, package in enumerate(all_bill_ids):
        print(num)
        bill_id = package["packageId"]
        # url = f"https://api.govinfo.gov/packages/{bill_id}/summary?api_key={API_KEY}"
        url = f'https://api.govinfo.gov/packages/{bill_id}/htm?api_key={API_KEY}'
        PARAMS = {'headers': 'accept: */*'}
        # PARAMS = {'headers': 'accept: application/json'}
        r = requests.get(url = url, params = PARAMS)
        # need to decode the bytes object
        # bill_summary = json.loads(r.content)
        # bill_text = json.loads(r.content)["download"]["txtLink"]
        if re.search(r"OVER_RATE_LIMIT", r.content.decode('utf8')):
            print("Reached limit at bill", bill_id)
            break
        all_bills[congress][bill_id] = r.content.decode('utf8')

    # NEEDS TO GO IN S3
    bucket_resource.put_object(Key=f'{congress}_bills_v{version}', Body=json.dumps(all_bills))
    print("check S3 again")
    print(len(all_bills[congress]), "bill texts gathered")
#     with open(f'bills/{congress}_bills.json', 'w') as outfile:
#         json.dump(all_bills, outfile)
    # return all_bills

In [46]:
API_KEY

'nt5nhSpwSqMbrGJ7hcsBkXFI1mfk80X0fexbnt45'

In [49]:
ids_115['115'][0]

{'packageId': 'BILLS-115s1866cps',
 'lastModified': '2021-03-25T18:34:33Z',
 'packageLink': 'https://api.govinfo.gov/packages/BILLS-115s1866cps/summary',
 'docClass': 's',
 'title': 'Hurricanes Harvey, Irma, and Maria Education Relief Act of 2017',
 'congress': '115',
 'dateIssued': '2017-09-26T12:00:00Z'}

In [50]:
url = f'https://api.govinfo.gov/packages/BILLS-115s1866cps/htm?api_key={API_KEY}'
PARAMS = {'headers': 'accept: */*'}
# PARAMS = {'headers': 'accept: application/json'}


In [51]:
r = requests.get(url = url, params = PARAMS)

In [55]:
r.content.decode('utf8')

"<html><body><pre>\n[Congressional Bills 115th Congress]\n[From the U.S. Government Publishing Office]\n[S. 1866 Considered and Passed Senate (CPS)]\n\n&lt;DOC&gt;\n\n\n\n\n\n\n115th CONGRESS\n  1st Session\n                                S. 1866\n\n  To provide the Secretary of Education with waiver authority for the \nreallocation rules and authority to extend the deadline by which funds \n   have to be reallocated in the campus-based aid programs under the \n Higher Education Act of 1965 due to Hurricane Harvey, Hurricane Irma, \n  and Hurricane Maria, to provide equitable services to children and \n          teachers in private schools, and for other purposes.\n\n\n_______________________________________________________________________\n\n\n                   IN THE SENATE OF THE UNITED STATES\n\n                           September 26, 2017\n\n  Mr. Alexander introduced the following bill; which was read twice, \n              considered, read the third time, and passed\n\n______

In [4]:
import re

In [ ]:
if re.search(r"OVER_RATE_LIMIT", r.content.decode('utf8')):
    break
    print("Reached limit at bill", package["packageId"])

In [5]:
response = s3.list_objects(Bucket='macs30123-bills')

In [7]:
response['Contents'][2]

{'Key': '116_bills',
 'LastModified': datetime.datetime(2021, 12, 3, 19, 45, 14, tzinfo=tzlocal()),
 'ETag': '"bd917adfbbea43a9d7193ee4e361fd06"',
 'Size': 52658781,
 'StorageClass': 'STANDARD',
 'Owner': {'DisplayName': 'awslabsc0w2817279t1632300722',
  'ID': '7d6595bfdb993ce297bc414145d8e4911db5a38be86efae6f310e6ff75aed196'}}

In [8]:
ids_116 = s3.get_object(Bucket='macs30123-bills', Key="116_ids")['Body'].read()

In [9]:
ids_116 = json.loads(ids_116)

In [11]:
url = f'https://api.govinfo.gov/packages/BILLS-116s1057cps/htm?api_key={API_KEY}'
PARAMS = {'headers': 'accept: */*'}
# PARAMS = {'headers': 'accept: application/json'}

In [12]:
r = requests.get(url = url, params = PARAMS)

In [16]:
print(re.search(r"OVER_RATE_LIMIT", "OVER_RATE_LIMIT"))

<re.Match object; span=(0, 15), match='OVER_RATE_LIMIT'>


In [15]:
print(re.search(r"OVER_RATE_LIMIT", r.content.decode('utf8')))

None


In [13]:
r.content.decode('utf8')

"<html><body><pre>\n[Congressional Bills 116th Congress]\n[From the U.S. Government Publishing Office]\n[S. 1057 Considered and Passed Senate (CPS)]\n\n&lt;DOC&gt;\n\n\n\n\n\n\n116th CONGRESS\n  1st Session\n                                S. 1057\n\n   To direct the Secretary of the Interior to execute and carry out \nagreements concerning Colorado River Drought Contingency Management and \n                  Operations, and for other purposes.\n\n\n_______________________________________________________________________\n\n\n                   IN THE SENATE OF THE UNITED STATES\n\n                             April 8, 2019\n\n    Ms. McSally (for herself, Ms. Cortez Masto, Mr. Barrasso, Mrs. \nFeinstein, Mr. Gardner, Mr. Romney, Ms. Sinema, Mr. Lee, Mr. Enzi, Ms. \nRosen, Mr. Bennet, Mr. Udall, Mr. Heinrich, and Ms. Harris) introduced \n the following bill; which was read twice, considered, read the third \n                            time, and passed\n\n_______________________________

In [19]:
len(ids_116['116'])

18614

In [32]:
ids_116['116'][602:]

[{'packageId': 'BILLS-116s1143is',
  'lastModified': '2021-03-25T17:57:10Z',
  'packageLink': 'https://api.govinfo.gov/packages/BILLS-116s1143is/summary',
  'docClass': 's',
  'title': 'Get the Lead Out of Military Housing Act',
  'congress': '116',
  'dateIssued': '2019-04-11T12:00:00Z'},
 {'packageId': 'BILLS-116s1142is',
  'lastModified': '2021-03-25T17:57:10Z',
  'packageLink': 'https://api.govinfo.gov/packages/BILLS-116s1142is/summary',
  'docClass': 's',
  'title': 'Energy Storage Tax Incentive and Deployment Act of 2019',
  'congress': '116',
  'dateIssued': '2019-04-11T12:00:00Z'},
 {'packageId': 'BILLS-116s1152is',
  'lastModified': '2021-03-25T17:57:09Z',
  'packageLink': 'https://api.govinfo.gov/packages/BILLS-116s1152is/summary',
  'docClass': 's',
  'title': 'To provide for the transfer of administrative jurisdiction over certain parcels of Federal land in Arlington, Virginia, and for other purposes.',
  'congress': '116',
  'dateIssued': '2019-04-11T12:00:00Z'},
 {'packag

In [45]:
602+604+602# +602

1808

In [51]:
import time

In [54]:
600*21

12600

In [57]:
API_KEY = 'XRdzNkSgsHhkZztYPLKRlzvPT0HBTnHdCezWpFQn'

In [59]:
# for num, x in enumerate(range(0, 600, 600)):
# for num, x in enumerate(range(0, 18614, 600)):
for num, x in enumerate(range(12600, 18614, 600)):
    print("version", num+21)
    get_package(ids_116['116'][x:x+600], "116", num+21)
    time.sleep(30)

version 21
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274

In [ ]:
for file in response['Contents']: 
    name = file['Key']
    print(name)
    if name.endswith('bills'): 
        obj = s3.get_object(Bucket='macs30123-bills', Key=name)['Body'].read()
        data = pd.read_json(obj) 
        data = from_pandas(data, npartitions=n_cores)
        break

Test that data is actually there

In [39]:
bills_115 = s3.get_object(Bucket="macs30123-bills", Key="115_bills")['Body'].read()

In [41]:
bills_115

In [42]:
bills115 = json.loads(bills_115)

In [45]:
bills115